In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from apyori import apriori
from mlxtend.frequent_patterns import fpgrowth
from mlxtend.frequent_patterns import association_rules
from mlxtend.preprocessing import TransactionEncoder

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [2]:
##ignora o codigo acima##

In [3]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from apyori import apriori
from mlxtend.frequent_patterns import fpgrowth
from mlxtend.frequent_patterns import association_rules
from mlxtend.preprocessing import TransactionEncoder

In [4]:
df = pd.read_csv('./compras.csv')
df = df.drop(columns=['cliente'])

In [5]:
df.stack().unique()

array(['limão', 'peito de peru', 'manga', 'cerveja', 'picanha', 'carvão',
       'sal', 'linguiça', 'mel', 'geleia', 'cereal', 'salsicha',
       'catchup', 'cebola', 'queijo', 'frutas', 'chá', 'alface',
       'granola', 'feijão', 'farinha de trigo', 'suco de laranja',
       'pimenta', 'sardinha enlatada', 'barra de cereal',
       'molho de tomate', 'gelatina', 'vinho branco', 'pão',
       'tempero pronto', 'vinho tinto', 'presunto', 'arroz', 'detergente',
       'filé de frango', 'papel higiênico', 'salada', 'cenoura',
       'maionese', 'molho branco', 'whisky', 'pudim', 'suco', 'azeitona',
       'frango', 'mostarda', 'carne moída', 'nhoque', 'água de coco',
       'laranja', 'cerveja artesanal', 'massa de pizza', 'biscoito',
       'morango', 'bisnaguinha', 'kiwi', 'achocolatado', 'sabão',
       'torrada', 'banana', 'palmito', 'desinfetante', 'sopa pronta',
       'ervilha', 'hambúrguer', 'batata frita', 'refrigerante',
       'queijo ralado', 'uva', 'óleo de soja', 'vodka', '

In [6]:
df.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17
0,limão,peito de peru,manga,cerveja,picanha,carvão,sal,linguiça,mel,geleia,cereal,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,salsicha,cerveja,picanha,catchup,geleia,cebola,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,queijo,frutas,chá,alface,granola,feijão,farinha de trigo,suco de laranja,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,pimenta,sardinha enlatada,barra de cereal,molho de tomate,gelatina,vinho branco,pão,tempero pronto,vinho tinto,presunto,arroz,feijão,NaN,NaN,NaN,NaN,NaN,NaN
4,detergente,filé de frango,arroz,feijão,limão,papel higiênico,presunto,farinha de trigo,salada,alface,cenoura,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [7]:
# Substituição dos valores
replace_dict = {'vinho tinto': 'vinho', 'vinho branco': 'vinho', 'alface': 'verdura'}
df.replace(to_replace=replace_dict, inplace=True)

# Encontrar valores únicos após a substituição
unique_values = df.stack().unique()

print(unique_values)


['limão' 'peito de peru' 'manga' 'cerveja' 'picanha' 'carvão' 'sal'
 'linguiça' 'mel' 'geleia' 'cereal' 'salsicha' 'catchup' 'cebola' 'queijo'
 'frutas' 'chá' 'verdura' 'granola' 'feijão' 'farinha de trigo'
 'suco de laranja' 'pimenta' 'sardinha enlatada' 'barra de cereal'
 'molho de tomate' 'gelatina' 'vinho' 'pão' 'tempero pronto' 'presunto'
 'arroz' 'detergente' 'filé de frango' 'papel higiênico' 'salada'
 'cenoura' 'maionese' 'molho branco' 'whisky' 'pudim' 'suco' 'azeitona'
 'frango' 'mostarda' 'carne moída' 'nhoque' 'água de coco' 'laranja'
 'cerveja artesanal' 'massa de pizza' 'biscoito' 'morango' 'bisnaguinha'
 'kiwi' 'achocolatado' 'sabão' 'torrada' 'banana' 'palmito' 'desinfetante'
 'sopa pronta' 'ervilha' 'hambúrguer' 'batata frita' 'refrigerante'
 'queijo ralado' 'uva' 'óleo de soja' 'vodka' 'batata' 'bolo' 'açúcar'
 'café' 'leite' 'atum enlatado' 'tomate' 'manteiga' 'vinagre' 'pepino'
 'molho pesto' 'água tônica' 'macarrão' 'peixe' 'energético' 'castanhas'
 'espumante' 'al

In [8]:
produtos_mais_comprados = df.stack().value_counts()
produtos_mais_comprados.head(30)

queijo               1241
vinho                1178
presunto             1129
pão                  1056
feijão               1040
picanha               869
arroz                 856
salada                837
manteiga              803
cerveja               759
leite                 722
bolo                  716
salsicha              682
açúcar                631
sorvete               621
verdura               618
carvão                612
batata frita          607
cerveja artesanal     581
tomate                577
refrigerante          573
frango                572
cenoura               554
azeite                522
catchup               521
vodka                 510
geleia                493
vinagre               479
mostarda              472
água tônica           433
Name: count, dtype: int64

In [9]:
df.replace(to_replace='cerveja artesanal', value='cerveja', inplace=True)
df.stack().unique()

array(['limão', 'peito de peru', 'manga', 'cerveja', 'picanha', 'carvão',
       'sal', 'linguiça', 'mel', 'geleia', 'cereal', 'salsicha',
       'catchup', 'cebola', 'queijo', 'frutas', 'chá', 'verdura',
       'granola', 'feijão', 'farinha de trigo', 'suco de laranja',
       'pimenta', 'sardinha enlatada', 'barra de cereal',
       'molho de tomate', 'gelatina', 'vinho', 'pão', 'tempero pronto',
       'presunto', 'arroz', 'detergente', 'filé de frango',
       'papel higiênico', 'salada', 'cenoura', 'maionese', 'molho branco',
       'whisky', 'pudim', 'suco', 'azeitona', 'frango', 'mostarda',
       'carne moída', 'nhoque', 'água de coco', 'laranja',
       'massa de pizza', 'biscoito', 'morango', 'bisnaguinha', 'kiwi',
       'achocolatado', 'sabão', 'torrada', 'banana', 'palmito',
       'desinfetante', 'sopa pronta', 'ervilha', 'hambúrguer',
       'batata frita', 'refrigerante', 'queijo ralado', 'uva',
       'óleo de soja', 'vodka', 'batata', 'bolo', 'açúcar', 'café',
       

In [10]:
produtos_mais_comprados = df.stack().value_counts()
produtos_mais_comprados.head(30)

cerveja         1340
queijo          1241
vinho           1178
presunto        1129
pão             1056
feijão          1040
picanha          869
arroz            856
salada           837
manteiga         803
leite            722
bolo             716
salsicha         682
açúcar           631
sorvete          621
verdura          618
carvão           612
batata frita     607
tomate           577
refrigerante     573
frango           572
cenoura          554
azeite           522
catchup          521
vodka            510
geleia           493
vinagre          479
mostarda         472
água tônica      433
óleo de soja     416
Name: count, dtype: int64

In [11]:
transactions = []
for i in range(0,df.shape[0]):
    transactions.append([str(df.values[i,j])for j in range(0,df.shape[1])])

regras = apriori(transactions=transactions, min_support=0.5, min_confidence=0.9, min_lift=3, min_length=2, max_length=2)

resultado = list(regras)
resultado

[]

In [12]:
df_compras_lista = df.apply(lambda x: x.dropna().tolist(), axis=1).tolist()

te = TransactionEncoder()
te_ary = te.fit(df_compras_lista).transform(df_compras_lista)

df_onehot = pd.DataFrame(te_ary, columns=te.columns_)

frequent_itemsets = fpgrowth(df_onehot, min_support=0.008,use_colnames=True)
rules = association_rules(frequent_itemsets, metric="lift", min_threshold=3)
rules = rules.sort_values(by='lift', ascending=False)
rules.head(50)

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction,zhangs_metric
209,"(cenoura, verdura)","(tomate, salada)",0.0129,0.0220,0.0083,0.643411,29.245948,0.008016,2.742652,0.978429
208,"(tomate, salada)","(cenoura, verdura)",0.0220,0.0129,0.0083,0.377273,29.245948,0.008016,1.585124,0.987533
108,"(manteiga, suco de laranja)","(bolo, pão)",0.0196,0.0152,0.0087,0.443878,29.202470,0.008402,1.770833,0.985064
109,"(bolo, pão)","(manteiga, suco de laranja)",0.0152,0.0196,0.0087,0.572368,29.202470,0.008402,2.292628,0.980662
211,"(verdura, tomate)","(cenoura, salada)",0.0160,0.0179,0.0083,0.518750,28.980447,0.008014,2.040727,0.981193
206,"(cenoura, salada)","(verdura, tomate)",0.0179,0.0160,0.0083,0.463687,28.980447,0.008014,1.834750,0.983091
210,"(cenoura, tomate)","(verdura, salada)",0.0126,0.0254,0.0083,0.658730,25.934258,0.007980,2.855805,0.973710
207,"(verdura, salada)","(cenoura, tomate)",0.0254,0.0126,0.0083,0.326772,25.934258,0.007980,1.466664,0.986498
331,(hambúrguer),"(batata frita, mostarda)",0.0348,0.0092,0.0081,0.232759,25.299850,0.007780,1.291380,0.995104
330,"(batata frita, mostarda)",(hambúrguer),0.0092,0.0348,0.0081,0.880435,25.299850,0.007780,8.072582,0.969392


In [13]:
rules = rules.sort_values(by='lift', ascending=True)
rules.head(50)

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction,zhangs_metric
51,(geleia),"(queijo, presunto)",0.0483,0.0576,0.0085,0.175983,3.055268,0.005718,1.143666,0.706837
46,"(queijo, presunto)",(geleia),0.0576,0.0483,0.0085,0.147569,3.055268,0.005718,1.116455,0.713812
377,(leite),(açúcar),0.0707,0.0624,0.0138,0.195191,3.128060,0.009388,1.164997,0.732070
376,(açúcar),(leite),0.0624,0.0707,0.0138,0.221154,3.128060,0.009388,1.193175,0.725590
295,(mostarda),(batata frita),0.0461,0.0600,0.0092,0.199566,3.326103,0.006434,1.174363,0.733146
294,(batata frita),(mostarda),0.0600,0.0461,0.0092,0.153333,3.326103,0.006434,1.126654,0.743987
415,(presunto),"(manteiga, pão)",0.1110,0.0397,0.0148,0.133333,3.358522,0.010393,1.108038,0.789933
410,"(manteiga, pão)",(presunto),0.0397,0.1110,0.0148,0.372796,3.358522,0.010393,1.417402,0.731282
484,(azeite),(verdura),0.0510,0.0607,0.0104,0.203922,3.359499,0.007304,1.179909,0.740081
485,(verdura),(azeite),0.0607,0.0510,0.0104,0.171334,3.359499,0.007304,1.145215,0.747723
